In [1]:
import sys
import os
sys.path.append('../')
os.chdir('../')

import pandas as pd
import numpy as np
import shutil
from pathlib import Path
from datetime import datetime
import time
import threading
import json
import requests
import threading
import re
from src import utils  


In [4]:
api_key = "577c8f76-500d-4e94-9139-8a798bdbe78f"
umls_url = "https://uts-ws.nlm.nih.gov/rest"
timeout_value = 10

In [9]:
def create_url(kind,list_var={}):

    """Dado el tipo de url, crea el link para realizar la peticion.

        Parametros
        ----------
            kind: String
                Tipo de petición a realizar en la API. Ex: Eutirox

        Retorna
        -------
            url: String
                Url para realizar la peticion, Ex: https://cima.aemps.es/cima/rest/medicamentos?&nombre=EUTIROX

    """

    url = umls_url + kind

    for index, (key, value) in enumerate(list_var.items()):
        if isinstance(value,int):
            url += "&" + key + "=" + str(value)
        elif isinstance(value,str):
            url += "&" + key + "=" + value
    print(url)
    return url

In [22]:
join_path = "/search/current?"

text = "trastorno gastrointestinal"

dic_var = {
    "string": text,
    "apiKey":api_key
}

url = create_url(join_path,dic_var)

params = {
    'pagina': 1,      # Setting the current page number
    'tamanioPagina': 25  # Setting the number of items per page
}

response = requests.get(url, params=params,timeout=timeout_value)
resp_code = response.status_code
resp_message = response.text

https://uts-ws.nlm.nih.gov/rest/search/current?&string=trastorno gastrointestinal&apiKey=577c8f76-500d-4e94-9139-8a798bdbe78f


In [23]:
resp_code

200

In [24]:
resp_message

'{"pageSize":25,"pageNumber":1,"result":{"classType":"searchResults","results":[{"ui":"C0017178","rootSource":"MTH","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0017178","name":"Gastrointestinal Diseases"},{"ui":"C0152945","rootSource":"MTH","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0152945","name":"Gastrointestinal anthrax"},{"ui":"C0236124","rootSource":"SNOMEDCT_US","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0236124","name":"Gastrointestinal obstruction"},{"ui":"C0161819","rootSource":"SNOMEDCT_US","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0161819","name":"Gastrointestinal complication"},{"ui":"C0237938","rootSource":"SNOMEDCT_US","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0237938","name":"Gastrointestinal ulcer"},{"ui":"C0079238","rootSource":"MTH","uri":"https://uts-ws.nlm.nih.gov/rest/content/2023AB/CUI/C0079238","name":"Digestive System Fistula"},{"ui":"C0852271","rootSource":"SNOMEDCT_US","uri":"htt